# Quantum Version of Fixed Strike Lookback Call Option

Can we approx lookback call using multiple vanilla european calls at different time steps?

Benefits to this approach:
- Can run multiple small quantum circuits (perhaps in parallel).
- Small circuits would be less error prone both in number of gates needed and total qubits on actual hardware.
  - For lots of timesteps, getting all of the values for each step could be impractical in total number of qubits (add however many are for prob dist plus add ancillas)
- Can simulate timesteps with different underlying uncertainty models fairly easily
  - Say the trend is different according to a holiday or season, so the prob model needs to change in those times
  
Drawbacks:
- Takes longer to run if not in parallel
- Can be a rough approximation, so not very accurate depending on time steps used
  - But so are "regular" quantum approaches (at least in "Option Pricing using Quantum Computers" from Stamatopoulos et al. and similar work)

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
from tqdm import tqdm

from qiskit import QuantumCircuit
from qiskit.algorithms import IterativeAmplitudeEstimation, EstimationProblem
from qiskit.circuit.library import LinearAmplitudeFunction
from qiskit_aer.primitives import Sampler
from qiskit_finance.circuit.library import LogNormalDistribution
from qiskit_finance.applications.estimation import EuropeanCallPricing

In [2]:
# Construct Uncertainty model helper function

def get_uncertainty_model(t, num_uncertainty_qubits=3, S=2.0, vol=0.4, r=0.0):
    # resulting parameters for log-normal distribution
    mu = (r - 0.5 * vol**2) * t + np.log(S)
    sigma = vol * np.sqrt(t)
    mean = np.exp(mu + sigma**2 / 2)
    variance = (np.exp(sigma**2) - 1) * np.exp(2 * mu + sigma**2)
    stddev = np.sqrt(variance)

    # lowest and highest value considered for the spot price; in between, an equidistant discretization is considered.
    low = np.maximum(0, mean - 3 * stddev)
    high = mean + 3 * stddev

    # construct A operator for QAE for the payoff function by
    # composing the uncertainty model and the objective
    uncertainty_model = LogNormalDistribution(
        num_uncertainty_qubits, mu=mu, sigma=sigma**2, bounds=(low, high)
    )
    return uncertainty_model, low, high

In [3]:
# Run Amplitude Estimation helper

def get_ae_results(uncert_model, low, high, uncert_q_count=3, K=1.896, scaling_factor=0.25, shots=100):
    # set target precision and confidence level
    epsilon = 0.01
    alpha = 0.05
    
    # Can save some lines of code by calling Qiskit Finance method here since they have a fast implementation set up for this option
    european_call_pricing = EuropeanCallPricing(
        num_state_qubits=uncert_q_count,
        strike_price=K,
        rescaling_factor=scaling_factor,
        bounds=(low, high),
        uncertainty_model=uncert_model,
    )
    problem = european_call_pricing.to_estimation_problem()
    # construct amplitude estimation
    ae = IterativeAmplitudeEstimation(
        epsilon_target=epsilon, alpha=alpha, sampler=Sampler(run_options={"shots": shots})
    )
    result = ae.estimate(problem)
    expected_payoff = european_call_pricing.interpret(result)
    conf_int = np.array(result.confidence_interval_processed)
    return expected_payoff, conf_int

In [4]:
# Run over the time steps, get maximum payoff
# Let's say 40 days, w/ time step = 1 day
T = 40 / 365
dt = 1 / 365
t = dt
results = []
conf_list = []
for i in range(40):
    um, low, high = get_uncertainty_model(t)
    res, conf = get_ae_results(um, low, high)
    results.append(res)
    conf_list.append(conf)
    t += dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Payoff 95% CI w/ eps 0.01: {conf_list[np.argmax(results)]}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.1672628054381453
Payoff 95% CI w/ eps 0.01: [0.16237729 0.17214833]
Lookback Option Day of Max Payoff: 40


Note that the above code typically selects a max payoff typically at or near the end of the time period (40th day). This is to be expected for this probability model (lognormal distribution in line with the Black Scholes model and its typical assumptions such as constant volatility). One would expect that given identical brownian motion procedures, the price of the underlying asset would most likely achieve a maximum at the end of the time period since it has had the most opportunity to grow in value. 

But what if the distributions for the stock value on each day are different? Whichever has the highest mean value is what one would expect to be selected for the lookback option. Let's look at this below.

In [5]:
# Now experiment with having say two consecutive days (e.g. a weekend/holiday) with a different uncertainty model
# To detect it with a lookback call, let's set days 21 & 22 to have 90% volatility (to have more room for value to go up/down)

T = 40 / 365
dt = 1 / 365
t = dt
results = []
conf_list = []
for i in range(40):
    if i == 20 or i == 21:  # These are really days 21 and 22
        um, low, high = get_uncertainty_model(t, vol=0.9)
    else:
        um, low, high = get_uncertainty_model(t)  # Still at default 40% vol
    res, conf = get_ae_results(um, low, high)
    conf_list.append(conf)
    results.append(res)
    t += dt
results = np.array(results)
print(f"Lookback Option Expected Payoff: {np.amax(results)}")
print(f"Payoff 95% CI w/ eps 0.01: {conf_list[np.argmax(results)]}")
print(f"Lookback Option Day of Max Payoff: {np.argmax(results)+1}")

Lookback Option Expected Payoff: 0.23661151492244598
Payoff 95% CI w/ eps 0.01: [0.2261618  0.24706123]
Lookback Option Day of Max Payoff: 22


Now that different distributions have been used, it can be more easily seen how a lookback options differs from a standard European option. It is not enough to just simulate the distrbution at the end of the maturity period. Therefore, either a large quantum circuit is needed, or several smaller circuits are needed (as shown here) to discretize the time periods. 